In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
import sys
sys.path.append('../')
from scripts.scrape_oldlistings import scrape_postcodes


In [2]:
scrape_postcodes()

HTTPError: 403 Client Error: Forbidden for url: https://www.onlymelbourne.com.au/melbourne-postcodes

In [5]:
sdf = spark.read.parquet("../data/landing/school_data/school_data.parquet")
sdf.show()

+--------------------+--------+--------------+---------------+
|         school_name|postcode|   school_type|education_level|
+--------------------+--------+--------------+---------------+
|Fintona Girls' Sc...|    3103|Non-government|      secondary|
|Melbourne High Sc...|    3141|    Government|      secondary|
|Mac.Robertson Gir...|    3004|    Government|      secondary|
|Nossal High Schoo...|    3806|    Government|      secondary|
|Suzanne Cory High...|    3030|    Government|      secondary|
|Presbyterian Ladi...|    3125|Non-government|      secondary|
|Ballarat Clarendo...|    3350|Non-government|      secondary|
|Fintona Girls' Sc...|    3103|Non-government|      secondary|
|  Haileybury College|    3173|Non-government|      secondary|
|Camberwell Gramma...|    3126|Non-government|      secondary|
|Camberwell Girls ...|    3126|Non-government|      secondary|
|Huntingtower Scho...|    3149|Non-government|      secondary|
|Korowa Anglican G...|    3146|Non-government|      sec

24/09/03 18:37:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7121443 ms exceeds timeout 120000 ms
24/09/03 18:37:43 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/03 18:37:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [13]:
# Create a DataFrame where each entry includes its URL
flattened_df = df.selectExpr("stack(" + str(len(df.columns)) + "," + ",".join([f"'{c}', `{c}`" for c in df.columns]) + ") as (id, properties)")

# Expand the properties into separate columns
expanded_df = flattened_df.select(
    "id",
    "properties.*"  # Expands the nested columns directly
)

AnalysisException: [DATATYPE_MISMATCH.STACK_COLUMN_DIFF_TYPES] Cannot resolve "stack(39, https://www.domain.com.au/1-36-clarendon-st-east-melbourne-vic-3002-16665538, https://www.domain.com.au/1-36-clarendon-st-east-melbourne-vic-3002-16665538, https://www.domain.com.au/1-tarver-st-port-melbourne-vic-3207-14253235, https://www.domain.com.au/1-tarver-st-port-melbourne-vic-3207-14253235, https://www.domain.com.au/101-beaconsfield-parade-albert-park-vic-3206-17000597, https://www.domain.com.au/101-beaconsfield-parade-albert-park-vic-3206-17000597, https://www.domain.com.au/11-peacock-street-brighton-vic-3186-17057578, https://www.domain.com.au/11-peacock-street-brighton-vic-3186-17057578, https://www.domain.com.au/14a-ethel-street-malvern-vic-3144-16398071, https://www.domain.com.au/14a-ethel-street-malvern-vic-3144-16398071, https://www.domain.com.au/15a-elizabeth-street-brighton-vic-3186-17183719, https://www.domain.com.au/15a-elizabeth-street-brighton-vic-3186-17183719, https://www.dom

In [8]:
# Extract all column names (URLs)
column_names = df.columns

schema = StructType([
    StructField("id", StringType(), True),
    StructField("url", StringType(), True),
    StructField("name", StringType(), True),
    StructField("cost_text", StringType(), True),
    StructField("rooms", ArrayType(StringType()), True),
    StructField("parking", ArrayType(StringType()), True),
    StructField("desc", StringType(), True),
    StructField("property_type", StringType(), True),
    StructField("date_available", StringType(), True),
    StructField("bond", StringType(), True),
    StructField("property_features", ArrayType(StringType()), True),
    StructField("coordinates", ArrayType(StringType()), True)
])

empty_df = spark.createDataFrame([], schema)

# Iterate over each column and add its data to the DataFrame
for column in column_names:
    new_rows = df.select(
        lit(column).alias("id"),
        col(f"{column}.url").alias("URL"),
        col(f"{column}.name").alias("name"),
        col(f"{column}.cost_text").alias("cost_text"),
        col(f"{column}.rooms").alias("rooms"),
        col(f"{column}.parking").alias("parking"),
        col(f"{column}.desc").alias("desc"),
        col(f"{column}.property_type").alias("property_type"),
        col(f"{column}.date_available").alias("date_available"),
        col(f"{column}.bond").alias("bond"),
        col(f"{column}.property_features").alias("property_features"),
        col(f"{column}.coordinates").alias("coordinates")
    )
    empty_df = empty_df.union(new_rows)

# Show the transformed DataFrame
empty_df.show(truncate=False)

+------------------------------------------------------+--------------------------------------------------------------------------------+----------------------------------------------+--------------------------------------------------+-----------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+--------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------